<h1 style="color: #492c68;">01 | BASICS</h1>

<h2 style="color: #327a81;">Libraries</h2>

In [1]:
## Basic libraries

import pandas as pd
import ast #
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## Settings

pd.set_option('display.max_columns', None) # display all columns
import warnings
warnings.filterwarnings('ignore') # ignore warnings

<h2 style="color: #327a81;">Data Read</h2>

In [2]:
data= pd.read_csv("netflix_mood_recommender.csv")

In [3]:
df = data.copy()

<h1 style="color: #492c68;">MOOD RECOMMENDER</h1>

- Let's finally build our mood recommender. We will code a way to ask the user for a kind of mood and we'll show suggestions from our titles

In [14]:
# First let's check a sample our final df

df.sample(2)

,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description,mood
443,s444,Movie,La ley de Herodes,Luis Estrada,"Damián Alcázar, Pedro Armendáriz Jr., Delia Ca...",Not Provided,2021-07-16,R,"Comedies, International Movies",NaN,123.0,Following the brutal 1949 murder of a small Me...,"disgust, anger"
571,s572,TV Show,Generation 56k,Not Provided,"Angelo Spagnoletti, Cristina Cappelli, Alfredo...",Not Provided,2021-07-01,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",1.0,NaN,A chance encounter leads two 30-somethings to ...,"surprise, joy"


In [55]:
df.shape

(8807, 13)

<h2 style="color: #327a81;">Mood one hot encoding</h2>

- Let's “hot code” the moods in the dataset so that our recommender works more efficiently.

In [5]:
# First we stack moods. One by one for each title

mood_encoding = df.drop(columns="mood").join(df["mood"].str.split(", ", expand=True).stack().reset_index(level=1, drop=True).rename("mood"))

In [6]:
# Then we apply "get dummies" to encode all mood possibilities

mood_encoding = pd.get_dummies(mood_encoding, columns=["mood"])*1

In [89]:
# Check out the encoding results

mood_encoding.head(2)

,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description,mood_anger,mood_disgust,mood_fear,mood_joy,mood_sadness,mood_surprise
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm...",0,0,0,0,1,0
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm...",0,0,0,1,0,0


In [90]:
# Filtering the obtained dataframe to show all encoded moods by show id

mood_columns = ["mood_anger", "mood_disgust", "mood_fear", "mood_joy", "mood_sadness", "mood_surprise"]
total_moods = mood_encoding.groupby("show_id")[mood_columns].sum()

# And merge the og dataframe with the encoded moods by show id

df_moods = df.merge(total_moods, on="show_id", how="inner")
df_moods

,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description,mood,mood_anger,mood_disgust,mood_fear,mood_joy,mood_sadness,mood_surprise
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm...","sadness, joy",0,0,0,1,1,0
1,s2,TV Show,Blood & Water,Not Provided,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,TV-MA,"International TV Shows, TV Dramas, TV Mysteries",2.0,NaN,"After crossing paths at a party, a Cape Town t...","disgust, fear",0,1,1,0,0,0
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Not Provided,2021-09-24,TV-MA,"Crime TV Shows, International TV Shows, TV Act...",1.0,NaN,To protect his family from a powerful drug lor...,"anger, disgust",1,1,0,0,0,0
3,s4,TV Show,Jailbirds New Orleans,Not Provided,Not Provided,Not Provided,2021-09-24,TV-MA,"Docuseries, Reality TV",1.0,NaN,"Feuds, flirtations and toilet talk go down amo...","disgust, joy",0,1,0,1,0,0
4,s5,TV Show,Kota Factory,Not Provided,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",2.0,NaN,In a city of coaching centers known to train I...,"disgust, fear",0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,2019-11-20,R,"Cult Movies, Dramas, Thrillers",NaN,158.0,"A political cartoonist, a crime reporter and a...","disgust, fear",0,1,1,0,0,0
8803,s8804,TV Show,Zombie Dumb,Not Provided,Not Provided,Not Provided,2019-07-01,TV-Y7,"Kids' TV, Korean TV Shows, TV Comedies",2.0,NaN,"While living alone in a spooky town, a young g...","joy, fear",0,0,1,1,0,0
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,R,"Comedies, Horror Movies",NaN,88.0,Looking to survive in a world taken over by zo...,"fear, disgust",0,1,1,0,0,0
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,PG,"Children & Family Movies, Comedies",NaN,88.0,"Dragged from civilian life, a former superhero...","disgust, fear",0,1,1,0,0,0


In [87]:
df_moods

,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description,mood,mood_anger,mood_disgust,mood_fear,mood_joy,mood_sadness,mood_surprise
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm...","sadness, joy",0,0,0,1,1,0
1,s2,TV Show,Blood & Water,Not Provided,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,TV-MA,"International TV Shows, TV Dramas, TV Mysteries",2.0,NaN,"After crossing paths at a party, a Cape Town t...","disgust, fear",0,1,1,0,0,0
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Not Provided,2021-09-24,TV-MA,"Crime TV Shows, International TV Shows, TV Act...",1.0,NaN,To protect his family from a powerful drug lor...,"anger, disgust",1,1,0,0,0,0
3,s4,TV Show,Jailbirds New Orleans,Not Provided,Not Provided,Not Provided,2021-09-24,TV-MA,"Docuseries, Reality TV",1.0,NaN,"Feuds, flirtations and toilet talk go down amo...","disgust, joy",0,1,0,1,0,0
4,s5,TV Show,Kota Factory,Not Provided,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",2.0,NaN,In a city of coaching centers known to train I...,"disgust, fear",0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,2019-11-20,R,"Cult Movies, Dramas, Thrillers",NaN,158.0,"A political cartoonist, a crime reporter and a...","disgust, fear",0,1,1,0,0,0
8803,s8804,TV Show,Zombie Dumb,Not Provided,Not Provided,Not Provided,2019-07-01,TV-Y7,"Kids' TV, Korean TV Shows, TV Comedies",2.0,NaN,"While living alone in a spooky town, a young g...","joy, fear",0,0,1,1,0,0
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,R,"Comedies, Horror Movies",NaN,88.0,Looking to survive in a world taken over by zo...,"fear, disgust",0,1,1,0,0,0
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,PG,"Children & Family Movies, Comedies",NaN,88.0,"Dragged from civilian life, a former superhero...","disgust, fear",0,1,1,0,0,0


<h2 style="color: #327a81;">Coding the recommender</h2>

In [101]:
def type_selection():
    while True:
        choice = input("Do you want to watch a Movie or a Tv Show?: ").strip().lower()
        if choice == "movie":
            type_mood = mood_encoding[mood_encoding["type"] == "Movie"]
            break
        elif choice == "tv show":
            type_mood =  mood_encoding[mood_encoding["type"] == "TV Show"]
            break
        else:
            print(f"{choice} is an invalid choice. Please choose either Movie or TV Show.")
            
    return type_mood

In [ ]:
def mood_selection():
    
    moods = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]
    
    mood1 = input(f"Select a mood according to what you want to see ({", ".join(moods)}): ").strip().lower()
    if mood1 not in moods:
        print(f"")
        

In [102]:
type_mood = type_selection()

Do you want to watch a Movie or a Tv Show?:  movies


movies is an invalid choice. Please choose either Movie or TV Show.


Do you want to watch a Movie or a Tv Show?:  Movie


In [103]:
type_mood

,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description,mood_anger,mood_disgust,mood_fear,mood_joy,mood_sadness,mood_surprise
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm...",0,0,0,0,1,0
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm...",0,0,0,1,0,0
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",Not Provided,2021-09-24,PG,Children & Family Movies,NaN,91.0,Equestria's divided. But a bright-eyed hero be...,0,0,0,1,0,0
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",Not Provided,2021-09-24,PG,Children & Family Movies,NaN,91.0,Equestria's divided. But a bright-eyed hero be...,1,0,0,0,0,0
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24,TV-MA,"Dramas, Independent Movies, International Movies",NaN,125.0,"On a photo shoot in Ghana, an American model s...",0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,R,"Comedies, Horror Movies",NaN,88.0,Looking to survive in a world taken over by zo...,0,1,0,0,0,0
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,PG,"Children & Family Movies, Comedies",NaN,88.0,"Dragged from civilian life, a former superhero...",0,1,0,0,0,0
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,PG,"Children & Family Movies, Comedies",NaN,88.0,"Dragged from civilian life, a former superhero...",0,0,1,0,0,0
8806,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,2019-03-02,TV-14,"Dramas, International Movies, Music & Musicals",NaN,111.0,A scrappy but poor boy worms his way into a ty...,0,0,1,0,0,0
